# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [38]:
#Your code here
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import norm

df=pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [39]:
df['action'].value_counts()

action
view     6328
click    1860
Name: count, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [40]:
contingency_table = pd.crosstab(df['group'], df['action'])

# Conduct chi-square test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)


# Print results
print("Chi-square statistic:", chi2)
print("P-value:", p_value)
print("Degrees of freedom:", dof)
print("Expected frequencies:\n", expected)

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("There is a significant association between the groups and actions.")
else:
    print("There is no significant association between the groups and actions.")

Chi-square statistic: 3.636160051233291
P-value: 0.056537191086915774
Degrees of freedom: 1
Expected frequencies:
 [[ 968.61748901 3295.38251099]
 [ 891.38251099 3032.61748901]]
There is no significant association between the groups and actions.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [41]:
#Your code here
# find the control click through rate
control = df[df['group']=='control']
ctrl_len = len(control)
control_click = control['action'].value_counts()['click']
control_ctr = control_click / len(control)
control_ctr

0.21857410881801126

In [42]:
# find the number of clicks for the control group

control_result = control_ctr *ctrl_len
control_result

932.0

In [43]:
df['group'].value_counts()

group
control       4264
experiment    3924
Name: count, dtype: int64

In [44]:
# find the number of clicks for the experiment group if it had the same CTR as the control group
experiment = df[df['group']=='experiment']
exp = len(experiment)
experiment_result = control_ctr * exp
experiment_result

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [45]:
#Your code here

# Step 2: Calculate number of standard deviations
variance_experiment = exp * control_ctr * (1 - control_ctr)
std_dev_experiment = np.sqrt(variance_experiment)
actual_clicks_experiment = experiment_result  # Observed number of clicks in the experiment group
z_score = (control_result - experiment_result) / std_dev_experiment
z_score

2.8705814726673333

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [47]:
#Your code here
# Step 3: Calculate p-value using normal distribution
p_value = 2 * (1 - norm.cdf(abs(z_score)))  # Multiply by 2 for two-tailed test
p_value

0.0040971759833776655

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: This does match what we previously saw from out rest rsults. Therefore, with such a low p-value, you would reject the null hypothesis and conclude that there is a statistically significant difference in the click-through rates between the two groups.

## My Analysis

A p-value of 0.0041 suggests that if the null hypothesis were true (i.e., there is no difference in click-through rates between the experiment and control groups), the probability of observing a difference as extreme as or more extreme than what was observed is only 0.41%.

Therefore, with such a low p-value, you would reject the null hypothesis and conclude that there is a statistically significant difference in the click-through rates between the two groups.






## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.